In [1]:
# Importations

import json
from datetime import datetime
import openrouteservice
import pandas as pd
import pyproj
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

In [2]:
def extract_coordinates_radius(dataframe, rad_col, lng_col, lat_col):
    
    """
    Extract coordinates and radius from a DataFrame.
    """
    
    radiuses, points_list = [], []

    for i, row in dataframe.iterrows():
        radiuses.append(row[rad_col])
        points_list.append((row[lng_col], row[lat_col]))
        
    return radiuses, points_list

In [3]:
def transform_coordinates(coord1, coord2, reverse=False):
    
    """
    Transform coordinates between different reference systems.
    """
    
    system1, system2 = "epsg:4326", "epsg:32632"
    if reverse:
        system1, system2 = system2, system1 
    transformer = pyproj.Transformer.from_crs(system1, system2)
    
    return transformer.transform(coord1, coord2)

In [4]:
def reverse_coordinates(coordinates_list):
    
    """
    Reverse coordinates order in a coordinates list.
    """
    
    return [(coor[1], coor[0]) for coor in coordinates_list]

In [5]:
def create_buffer_polygon(point, radius, resolution=1):
    
    """
    Create a polygon around a point from a given radius.
    """
    
    point_utm = transform_coordinates(*point)
    point_buffer = Point(point_utm).buffer(distance = radius, 
                                           resolution = resolution)
    polygon_points = []
    for point in point_buffer.exterior.coords:
        polygon_points.append(transform_coordinates(*point, reverse = True))

    return polygon_points

In [6]:
def style_function(color):
    
    """
    A useful function to generate different color styles.
    """
    
    return lambda feature: dict(color = color, weight = 3, opacity = 0.5)

In [7]:
def get_normal_route(client, coordinates):
    
    """
    Calculate the route between two points given by their coordinates.
    """
    
    normal_route = client.directions(coordinates,
                                     profile = "foot-walking",
                                     preference = "recommended",
                                     format_out = "geojson",
                                     geometry_simplify = True,
                                     dry_run = False)
    
    return normal_route

In [8]:
def get_buffer_route(normal_route):
    
    """
    Calculate the influence area around the route between two points.
    """
    
    route = LineString(normal_route["features"][0]["geometry"]["coordinates"])
    buffer_route = route.buffer(0.005)
    
    return buffer_route

In [9]:
def obtain_polygons(route_buffer, points_list, radiuses):
    
    """
    Generate the polygons that are present in an influence area.
    """
    
    sites_polygon = []
    
    for site_coordinates, radius in zip(points_list, radiuses):
        if route_buffer.intersects(Point(site_coordinates)):
            site_polygon_coordinates = create_buffer_polygon(site_coordinates, 
                                                             radius = radius)
            sites_polygon.append(Polygon(site_polygon_coordinates))
        
    return sites_polygon

In [10]:
def get_detour_route(client, coordinates, buffer_route, sites_polygon):
    
    """
    Get the best route between two points avoiding forbidden polygons.
    """
    
    request_params = {"coordinates": coordinates,
                      "profile": "foot-walking",
                      "preference": "recommended",
                      "format_out": "geojson",
                      "options": {
                          "avoid_polygons": geometry.mapping(MultiPolygon(sites_polygon))
                      },
                      "geometry_simplify": True,
                      "dry_run": False}
    
    route_detour = client.directions(**request_params)
    
    return route_detour

In [11]:
def get_detour_route_coordinates(detour_route):
    
    """
    Extract the coordinates of the best route calculated.
    """
    
    coor = detour_route["features"][0]["geometry"]["coordinates"]
    
    return coor

In [12]:
def create_route_json(coordinates, dtype = "list"):
    
    """
    Create a json object with a route coordinates list or dict.
    """
    
    if dtype == "dict":
        coordinates = [[element["lng"], element["lat"]] for element in coordinates]
    elif dtype == "list":
        coordinates = [[element[0], element[1]] for element in coordinates]
        
    route = {"type": "Feature",
             "geometry": {"type": "LineString", 
                          "coordinates": coordinates}}
    return route

In [13]:
def main(coordinates):
    
    """
    The main function to get the best route between two points.
    """
    
    API_KEY = "5b3ce3597851110001cf62486d267b042a754245aeb826068a8ea035"
    client = openrouteservice.Client(key=API_KEY)
    coords = (coordinates[0], coordinates[1])
    df = pd.read_csv("points.csv")
    radiuses, points_list = extract_coordinates_radius(dataframe = df, 
                                                       rad_col = "radio(m)", 
                                                       lng_col = "Longitude",
                                                       lat_col = "Latitude")
    normal_route = get_normal_route(client, coords)
    buffer_route = get_buffer_route(normal_route)
    sites_polygon = obtain_polygons(buffer_route, points_list, radiuses)
    route_detour = get_detour_route(client, coords, buffer_route, sites_polygon)
    route_coordinates = get_detour_route_coordinates(route_detour)
    route_coordinates.insert(0, list(coords[0]))
    route_coordinates.append(list(coords[1]))
    
    return create_route_json(route_coordinates, dtype="list")

In [14]:
def parse(input_info):
    
    """
    A function to parse the given information in an specific format.
    """
    
    parameters = input_info.split("&")
    parsed_info = {parameter.split("=")[0]: parameter.split("=")[1] \
                   for parameter in parameters}
    origin_info = parsed_info["origin"].split(",")
    origin = (float(origin_info[0]), float(origin_info[1]))

    destination_info = parsed_info["destination"].split(",")
    destination = (float(destination_info[0]), float(destination_info[1]))

    coordinates = (origin, destination)

    return coordinates

In [15]:
### EXAMPLE ###

In [16]:
coords = parse("origin=-3.714517,40.433050&destination=-3.708298,40.438797")
coor = main(coords)
coor

{'type': 'Feature',
 'geometry': {'type': 'LineString',
  'coordinates': [[-3.714517, 40.43305],
   [-3.714517, 40.433055],
   [-3.710347, 40.432923],
   [-3.710198, 40.435266],
   [-3.709939, 40.438747],
   [-3.709856, 40.438747],
   [-3.709855, 40.438758],
   [-3.70839, 40.438682],
   [-3.708381, 40.4388],
   [-3.708298, 40.438797],
   [-3.708298, 40.438797]]}}

In [17]:
import folium

coords = coor["geometry"]["coordinates"]

map = folium.Map(location = [40.433050, -3.714517],
                 tiles='Stamen Toner',
                 zoom_start = 16)

for point in coords:
    folium.Marker(reverse_coordinates([point])[0]).add_to(map)

map